## Functions Definition

In [ ]:
def dwi_denoise(brain_path,dwi_name,b_name,b0_min,para,method,data_out):
    '''
    Study_folder
    tau_factor=para[1], patch_radius=para[2],pca_method='svd'
    '''
    from time import time
    from dipy.denoise.localpca import localpca
    from dipy.denoise.pca_noise_estimate import pca_noise_estimate
    from dipy.core.gradients import gradient_table
    import nipype.interfaces.fsl as fsl #topup
    from nipype.interfaces.fsl import TOPUP
    from nipype.interfaces.fsl import ApplyTOPUP
    from nipype.interfaces.fsl import Eddy
    from nipype.interfaces.fsl import ExtractROI
    import numpy as np
    import nibabel as nib
    import os
    from nibabel.nifti1 import Nifti1Header
    from tqdm.notebook import tqdm
    from dipy.io.image import load_nifti, save_nifti
    from dipy.io import read_bvals_bvecs
    import pandas as pd

    for Study_folder in tqdm(brain_path):
        os.chdir(Study_folder)
        data_prefix = dwi_name
        dwi_file = os.path.join(Study_folder,data_prefix+'.nii') # 4D diffusion data file name
        img_data,img_affine,img = load_nifti(dwi_file,return_img=True)
        bval = os.path.join(Study_folder,b_name+'.bval') # bval file name
        bvec = os.path.join(Study_folder,b_name+'.bvec') # bvec file name
        #------------
        bvals, bvecs = read_bvals_bvecs(bval,bvec)
        gtab = gradient_table(bvals, bvecs,atol=0.15,b0_threshold=b0_min)

        data_undenoise = img_data
        if para == None:
            para = [3,2.3,3]
        else:
            para = para
        t = time()
        sigma = pca_noise_estimate(data_undenoise, gtab, correct_bias=True, smooth=para[0])
        # save sigma
        save_nifti('sigma_3d.nii', sigma.astype(np.float32), img_affine,hdr=img.header)
        print("Sigma estimation time", time() - t)
        t = time()
        if method == None:
            p_method = 'svd'
        else:
            p_method = method
        denoised_arr = localpca(data_undenoise, sigma, tau_factor=para[1], patch_radius=para[2],pca_method=p_method)
        print("Time taken for local PCA", -t + time())
        # save results
        save_nifti(data_out+'.nii', denoised_arr.astype(np.int16), img_affine,hdr=img.header)
        print("Done")

In [ ]:
def snr_cal(brain_path,dwi_name,b_name,mask,b0_min,threshold_rgb):
    '''
    Study_folder
    '''
    import numpy as np
    from dipy.core.gradients import gradient_table
    from dipy.data import get_fnames
    from dipy.io.gradients import read_bvals_bvecs
    from dipy.io.image import load_nifti, save_nifti
    from dipy.segment.mask import median_otsu
    from dipy.reconst.dti import TensorModel
    from time import time
    from dipy.denoise.localpca import localpca
    from dipy.denoise.pca_noise_estimate import pca_noise_estimate
    from dipy.core.gradients import gradient_table
    import nipype.interfaces.fsl as fsl #topup
    from nipype.interfaces.fsl import TOPUP
    from nipype.interfaces.fsl import ApplyTOPUP
    from nipype.interfaces.fsl import Eddy
    from nipype.interfaces.fsl import ExtractROI
    import numpy as np
    import nibabel as nib
    import os
    from nibabel.nifti1 import Nifti1Header
    from tqdm.notebook import tqdm
    from dipy.io.image import load_nifti, save_nifti
    from dipy.io import read_bvals_bvecs
    import pandas as pd
    from dipy.segment.mask import segment_from_cfa
    from dipy.segment.mask import bounding_box
    import matplotlib.pyplot as plt
    from scipy.ndimage.morphology import binary_dilation
    
    for Study_folder in tqdm(brain_path):
        os.chdir(Study_folder)
        data_prefix = dwi_name
        dwi_file = os.path.join(Study_folder,data_prefix+'.nii') # 4D diffusion data file name
        data,affine,img = load_nifti(dwi_file,return_img=True)
        mask_file = os.path.join(Study_folder,mask+'.nii') # 4D diffusion data file name
        mask,mask_affine,mask_img = load_nifti(mask_file,return_img=True)
        bval = os.path.join(Study_folder,b_name+'.bval') # bval file name
        bvec = os.path.join(Study_folder,b_name+'.bvec') # bvec file name
        #------------
        bvals, bvecs = read_bvals_bvecs(bval,bvec)
        gtab = gradient_table(bvals, bvecs,atol=0.05,b0_threshold=b0_min)
        # generate rgb maps
        tenmodel = TensorModel(gtab)
        tensorfit = tenmodel.fit(data, mask=mask)
        # find the red region (CC) after generating rgb maps
        print('Computing worst-case/best-case SNR using the corpus callosum...')
        if threshold_rgb == None:
            threshold = (0.3, 1, 0, 0.26, 0, 0.26)
        else:
            threshold = threshold_rgb
        CC_box = np.zeros_like(data[..., 0])
        mins, maxs = bounding_box(mask)
        mins = np.array(mins)
        maxs = np.array(maxs)
        print(mins,maxs)
        diff = (maxs - mins) // 4
        bounds_min = mins + diff
        bounds_max = maxs - diff
        # this is to find the brain tissue bounds
        CC_box[bounds_min[0]:bounds_max[0],
            bounds_min[1]:bounds_max[1],
            bounds_min[2]:bounds_max[2]] = 1

        mask_cc_part, cfa = segment_from_cfa(tensorfit, CC_box, threshold,
                                            return_cfa=True)
        zooms_old = img.header.get_zooms()[:4]
        img_new = nib.Nifti1Image(cfa.astype(np.uint8), np.eye(4))
        img_new.header.set_zooms(zooms_old[0:4])
        save_nifti('cfa_CC_part.nii', (cfa*255).astype(np.uint8), affine,hdr=img_new.header)
        img_new = nib.Nifti1Image(mask_cc_part.astype(np.uint8), np.eye(4))
        img_new.header.set_zooms(zooms_old[0:3])
        save_nifti('mask_CC_part.nii', mask_cc_part.astype(np.uint8), affine,hdr=img_new.header)
        # plot cc region found
        region = int(mask.shape[0]//2)
        fig = plt.figure('Corpus callosum segmentation')
        plt.subplot(1, 2, 1)
        plt.title("Corpus callosum (CC)")
        plt.axis('off')
        red = cfa[..., 0]
        plt.imshow(np.rot90(red[region,:,:]))
        plt.subplot(1, 2, 2)
        plt.title("CC mask used for SNR computation")
        plt.axis('off')
        plt.imshow(np.rot90(mask_cc_part[region, ...]))
        fig.savefig("CC_segmentation.png", bbox_inches='tight',dpi=400)
        # 
        mean_signal = np.mean(data[mask_cc_part], axis=0)
        mask_noise = binary_dilation(mask, iterations=10)
        mask_noise[..., :mask_noise.shape[-1]//2] = 1
        mask_noise = ~mask_noise
        #
        save_nifti('mask_noise.nii', mask_noise.astype(np.uint8), affine,hdr=img_new.header)
        noise_std = np.std(data[mask_noise, :])
        print('Noise standard deviation sigma= ', noise_std)

        # Exclude null bvecs from the search
        idx = np.sum(gtab.bvecs, axis=-1) == 0
        gtab.bvecs[idx] = np.inf
        axis_X = np.argmin(np.sum((gtab.bvecs-np.array([1, 0, 0]))**2, axis=-1))
        axis_Y = np.argmin(np.sum((gtab.bvecs-np.array([0, 1, 0]))**2, axis=-1))
        axis_Z = np.argmin(np.sum((gtab.bvecs-np.array([0, 0, 1]))**2, axis=-1))

        snr_save = np.zeros(5)# first four are snrs, the last is standard deviation
        snr_save[4] = noise_std
        i_snr = -1
        for direction in [0, axis_X, axis_Y, axis_Z]:
            i_snr = i_snr +1
            SNR = mean_signal[direction]/noise_std
            if direction == 0:
                print("SNR for the b=0 image is :", SNR)
            else:
                print("SNR for direction", direction, " ",
                    gtab.bvecs[direction], "is :", SNR)
            snr_save[i_snr] = SNR
        np.savetxt('snr_'+data_prefix,snr_save,fmt='%1.2f')

In [ ]:
def snr_display(brain_path,snr1,snr2,label1,label2,title):
    '''
    '''
    import matplotlib.pylab as plt
    import numpy as np
    import os
    from tqdm import tqdm
    for Study_folder in tqdm(brain_path):
        os.chdir(Study_folder)
        if snr2 != None:
            snr_0 = np.loadtxt(os.path.join(Study_folder,snr1))
            snr_d = np.loadtxt(os.path.join(Study_folder,snr2))
            plt.figure()
            plt.subplot()
            plt.bar(np.arange(len(snr_d)),height=snr_0,width=0.4,label=label1)
            for i_text in np.arange(len(snr_0)):
                plt.text(np.arange(len(snr_0))[i_text]-0.2,snr_0[i_text],snr_0[i_text])

            plt.bar(np.arange(len(snr_d))+0.4,height=snr_d,width=0.4,label=label2)
            for i_text in np.arange(len(snr_d)):
                plt.text(np.arange(len(snr_d))[i_text]+0.2,snr_d[i_text],snr_d[i_text])

            plt.xticks(np.arange(len(snr_d))+0.2,['b0','x','y','z','std(noise)'])
            plt.ylabel('SNR (or values)')
            plt.legend()
            plt.savefig(title+'.png',dpi=200,facecolor='w', edgecolor='w',bbox_inches=None,pad_inchies=None,transparent=False)
            plt.show()
        else:
            snr_0 = np.loadtxt(snr1)
            plt.figure()
            plt.subplot()
            plt.bar(np.arange(len(snr_0)),height=snr_0,width=0.4,label=label1)
            for i_text in np.arange(len(snr_0)):
                plt.text(np.arange(len(snr_0))[i_text]-0.2,snr_0[i_text],snr_0[i_text])
            plt.xticks(np.arange(len(snr_0)),['b0','x','y','z','std(noise)'])
            plt.ylabel('SNR (or values)')
            plt.legend()
            plt.savefig(title+'.png',dpi=200,facecolor='w', edgecolor='w',bbox_inches=None,pad_inchies=None,transparent=False)
            plt.show()

In [ ]:
# Diffusion1 denoising
brain_path = brain_path = ['/media/erjun/One_Touch/Newborns/sub-B027S1/ses-01/Diffusion2/']
dwi_denoise(brain_path,dwi_name='dwi_mb0',b_name='dwi_mb0',
            b0_min=5,para=None,method=None,data_out='dwi_d')